In [1]:
import numpy as np
import pandas as pd


In [2]:
dataDir = '/Users/anupama/Documents/Instacart/'

In [3]:
priors = pd.read_csv(dataDir + 'order_products__prior.csv')
priors[:1]

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1


In [4]:
train = pd.read_csv(dataDir + 'order_products__train.csv')
train[:1]

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1


In [5]:
frames=[priors, train]
prior_train = pd.concat(frames, ignore_index=True)
prior_train[:1]

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1


In [6]:
orders = pd.read_csv(dataDir + 'orders.csv')


In [7]:
orders.fillna(0, inplace=True)
orders[:2]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,15.0


In [8]:
orders.nunique()

order_id                  3421083
user_id                    206209
eval_set                        3
order_number                  100
order_dow                       7
order_hour_of_day              24
days_since_prior_order         31
dtype: int64

In [9]:
departments = pd.read_csv(dataDir  + 'departments.csv')

In [10]:
aisles = pd.read_csv(dataDir + 'aisles.csv')

In [11]:
products = pd.read_csv(dataDir + 'products.csv')
products[:1]

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19


### Product level feature engineering
- **product_num_orders :** total number of orders per product
- **reorders :** total number of reorders per product
- **reordered_rate :** product reordered rate
- **Add to cart order stats:** mean,median,min,max,std,sum stats for add_to_cart_order per product

In [12]:
##product feature engineering
prods = pd.DataFrame()
prods['product_num_orders'] = prior_train.groupby('product_id').size().astype(np.int16)
prods['reorders'] = prior_train.groupby('product_id')['reordered'].sum().astype(np.float32)
prods['reordered_rate'] = prods['reorders']/prods['product_num_orders'].astype(np.float32)
prods['mean_pos_in_cart'] = prior_train.groupby('product_id')['add_to_cart_order'].mean().astype(np.float32)
prods['min_pos_in_cart'] = prior_train.groupby('product_id')['add_to_cart_order'].min().astype(np.int32)
prods['max_pos_in_cart'] = prior_train.groupby('product_id')['add_to_cart_order'].max().astype(np.int32)
prods['std_pos_in_cart'] = prior_train.groupby('product_id')['add_to_cart_order'].std().astype(np.float32)
prods['sum_pos_in_cart'] = prior_train.groupby('product_id')['add_to_cart_order'].sum().astype(np.int32)
prods = prods.reset_index()

In [13]:
prods[:1]

,product_id,product_num_orders,reorders,reordered_rate,mean_pos_in_cart,min_pos_in_cart,max_pos_in_cart,std_pos_in_cart,sum_pos_in_cart
0,1,1928,1185.0,0.614627,5.845954,1,74,5.648925,11271


In [14]:

prior_train_order = pd.merge(prior_train, orders, on='order_id', how = 'inner')
len(prior_train_order)

33819106

In [15]:
usr_reordered_cnt = prior_train_order.groupby('user_id')['reordered'].sum().astype(np.int16)
usr_reordered_cnt[:5]

user_id
1     51
2    105
3     55
4      1
5     18
Name: reordered, dtype: int16

### User level feature engineering
- **u_num_orders:** Number of orders per user
- **u_total_items:** Total number of items per user
- **u_avg_basket_size:** Average basket size per user
- **u_reordered_cnt:** Reorder count per user
- **u_reordered_rate:** User reordered rate
- **u_days_since_prior_order:** Days_since_prior_order stats: mean, median,min,max,std dev
- **u_distinct_items_count:** Number of distinct items per user
- **u_non_reordered_cnt:** Number of products that were not reordered per user
- **u_organic_ratio:** Ratio of organic products purchased for each user

In [16]:
##user feature engineering
user = pd.DataFrame()
user['u_num_orders'] = orders.groupby('user_id').size().astype(np.int16)
user['u_total_items'] = prior_train_order.groupby('user_id').size().astype(np.int16)
user['u_avg_basket_size'] = user['u_total_items']/user['u_num_orders'].astype(np.float32)
#basket size stats min, max, std
user['u_reordered_cnt'] = prior_train_order.groupby('user_id')['reordered'].sum().astype(np.int16)
user['u_reordered_rate'] = user['u_reordered_cnt']/user['u_total_items'].astype(np.float32)
user['u_days_since_prior_order_mean'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
user['u_days_since_prior_order_min'] = orders.groupby('user_id')['days_since_prior_order'].min().astype(np.int32)
user['u_days_since_prior_order_max'] = orders.groupby('user_id')['days_since_prior_order'].max().astype(np.int32)
user['u_days_since_prior_order_sd'] = orders.groupby('user_id')['days_since_prior_order'].std().astype(np.float32)
user['u_distinct_items_cnt'] = prior_train_order.groupby('user_id')['product_id'].nunique().astype(np.int16)

temp = pd.DataFrame()
temp['u_product_reordered_cnt'] = prior_train_order.groupby(['user_id','product_id'])['reordered'].sum().astype(np.int16)
temp['u_non_reordered_cnt'] = temp[temp.u_product_reordered_cnt==0].astype(np.int16)
usr = temp.groupby('user_id').count().astype(np.int32)
usr.drop('u_product_reordered_cnt',inplace=True, axis=1)

user = user.join(usr)



In [17]:
#user.info()

In [18]:
products['isOrganic'] = products.product_name.map(lambda x: 'organic' in x.lower())*1

In [19]:
merged_products_temp = pd.merge(products, departments, on='department_id', how = 'inner')
merged_products = pd.merge(merged_products_temp, aisles, on='aisle_id', how = 'inner')
merged_all = pd.merge(prior_train_order, merged_products, on='product_id', how='inner')
merged_all[:1]

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,isOrganic,department,aisle
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1,dairy eggs,eggs


In [20]:

user['u_organic_ratio'] = merged_all.groupby('user_id')['isOrganic'].mean().astype(np.float32)


In [21]:
user[:5]


,u_num_orders,u_total_items,u_avg_basket_size,u_reordered_cnt,u_reordered_rate,u_days_since_prior_order_mean,u_days_since_prior_order_min,u_days_since_prior_order_max,u_days_since_prior_order_sd,u_distinct_items_cnt,u_non_reordered_cnt,u_organic_ratio
user_id,,,,,,,,,,,,
1,11,70,6.363636,51,0.728571,17.272728,0,30,10.306221,19,7,0.257143
2,15,226,15.066667,105,0.464602,15.200000,0,30,10.751744,121,79,0.247788
3,13,88,6.769231,55,0.625000,11.076923,0,21,5.936631,33,14,0.340909
4,6,18,3.000000,1,0.055556,14.166667,0,30,12.023588,17,16,0.111111
5,5,46,9.200000,18,0.391304,9.200000,0,19,6.978539,28,18,0.500000


### User_Product feature engineering
- **up_product_count:** number of times each product was ordered by user
- **up_product_purchase_rate:** rate at which each product was ordered for the user
- **up_product_reorder_count:** number of times each product was ordered for each user
- **up_product_last_order_num:** the order number when the product was ordered last
- **up_product_order_num_diff:** how many times has the product been skipped since the last order
- **up_add_to_cart_order:** add_to_cart_order stats per product per user (mean, median, min, max, std dev, sum)
- **up_reorder_probability:** each product's reorder probability per user

In [22]:
user_product = pd.DataFrame()
user_product['up_product_count'] = merged_all.groupby(['user_id','product_id']).size().astype(np.int16)
user_product['up_product_purchase_rate'] = user_product['up_product_count']/user['u_num_orders'].astype(np.float32)
user_product['up_product_reorder_count'] = merged_all.groupby(['user_id','product_id'])['reordered'].sum().astype(np.int16)



In [23]:
user_product['up_product_last_order_num'] = merged_all.groupby(['user_id','product_id'])['order_number'].max().astype(np.int16)
user_product['up_product_order_num_diff'] = user['u_num_orders'] - user_product['up_product_last_order_num'].astype(np.int16)
user_product['up_add_to_cart_order_mean'] = merged_all.groupby(['user_id','product_id'])['add_to_cart_order'].mean().astype(np.float16)
user_product['up_add_to_cart_order_min'] = merged_all.groupby(['user_id','product_id'])['add_to_cart_order'].min().astype(np.int16)
user_product['up_add_to_cart_order_max'] = merged_all.groupby(['user_id','product_id'])['add_to_cart_order'].max().astype(np.int16)
user_product['up_add_to_cart_order_sd'] = merged_all.groupby(['user_id','product_id'])['add_to_cart_order'].std().astype(np.float16)
user_product['up_add_to_cart_order_sum'] = merged_all.groupby(['user_id','product_id'])['add_to_cart_order'].sum().astype(np.int16)


In [24]:
#not sure if I should do this
usr1 = pd.DataFrame()
usr1['max_purchase_rate'] = user_product.groupby('user_id', as_index=True)['up_product_purchase_rate'].max().astype(np.float32)
user_product['up_reorder_probability'] = user_product['up_product_purchase_rate']/usr1['max_purchase_rate'].astype(np.float32)


In [25]:
user = user.reset_index()
prods = prods.reset_index()
user_product = user_product.reset_index()

# SOLUTION 1

In [26]:
user_product_list=pd.DataFrame()
user_product_list['all_products'] = user_product.groupby('user_id')['product_id'].apply(set)
#user_product_list=user_product_list.reset_index()
user_product_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 1 to 206209
Data columns (total 1 columns):
all_products    206209 non-null object
dtypes: object(1)
memory usage: 3.1+ MB


In [27]:
orders_train = orders[(orders.eval_set=='train')]
orders_test = orders[orders.eval_set=='test']

prior_train.set_index(['order_id','product_id'], inplace=True, drop=False)
prior_train_index = set(prior_train.index)


In [28]:

def construct_features(df, create_labels = False):
    order_list = []
    product_list = []
    labels = []

    for row in df.itertuples():   
    
        user_id = row.user_id
        order_id = row.order_id
    
        user_prods = user_product_list['all_products'][user_id]
        product_list += user_prods
        order_list += [order_id]*len(user_prods)
        if(create_labels):
            labels += [(order_id, prod) in prior_train_index for prod in user_prods]
    
    
    final_df = pd.DataFrame({'order_id':order_list, 'product_id':product_list, 'label':labels}, dtype=np.int32)
    ##product feature engineering
    
    final_df['product_num_orders'] = final_df.product_id.map(prods.product_num_orders)
    final_df['reorders'] = final_df.product_id.map(prods.reorders)
    final_df['reordered_rate'] = final_df.product_id.map(prods.reordered_rate)
    final_df['mean_pos_in_cart'] = final_df.product_id.map(prods.mean_pos_in_cart)
    final_df['min_pos_in_cart'] = final_df.product_id.map(prods.min_pos_in_cart)
    final_df['max_pos_in_cart'] = final_df.product_id.map(prods.max_pos_in_cart)
    final_df['std_pos_in_cart'] = final_df.product_id.map(prods.std_pos_in_cart)
    final_df['sum_pos_in_cart'] = final_df.product_id.map(prods.sum_pos_in_cart)
    
    
    final_df['user_id'] = final_df.order_id.map(orders.user_id)
    final_df['u_num_orders'] = final_df.user_id.map(user.u_num_orders)
    final_df['u_total_items'] = final_df.user_id.map(user.u_total_items)
    final_df['u_avg_basket_size'] = final_df.user_id.map(user.u_avg_basket_size)
    final_df['u_reordered_cnt'] = final_df.user_id.map(user.u_reordered_cnt)
    final_df['u_reordered_rate'] = final_df.user_id.map(user.u_reordered_rate)
    final_df['u_days_since_prior_order_mean'] = final_df.user_id.map(user.u_days_since_prior_order_mean)
    final_df['u_days_since_prior_order_min'] = final_df.user_id.map(user.u_days_since_prior_order_min)
    final_df['u_days_since_prior_order_max'] = final_df.user_id.map(user.u_days_since_prior_order_max)
    final_df['u_days_since_prior_order_sd'] = final_df.user_id.map(user.u_days_since_prior_order_sd)
    final_df['u_distinct_items_cnt'] = final_df.user_id.map(user.u_distinct_items_cnt)
    final_df['u_organic_ratio'] = final_df.user_id.map(user.u_organic_ratio)
    final_df['u_non_reordered_cnt'] = final_df.user_id.map(user.u_non_reordered_cnt)
        
    return final_df

In [29]:
train_df = construct_features(orders_train, create_labels = True)

In [30]:
train_df.shape()


(9030454, 24)

In [31]:
train_df[:1]

,order_id,product_id,label,product_num_orders,reorders,reordered_rate,mean_pos_in_cart,min_pos_in_cart,max_pos_in_cart,std_pos_in_cart,...,u_avg_basket_size,u_reordered_cnt,u_reordered_rate,u_days_since_prior_order_mean,u_days_since_prior_order_min,u_days_since_prior_order_max,u_days_since_prior_order_sd,u_distinct_items_cnt,u_organic_ratio,u_non_reordered_cnt
0,1187899,17122,0,114.0,49.0,0.429825,8.929825,1.0,33.0,6.646204,...,11.782609,188.0,0.693727,8.304348,0.0,30.0,6.858537,83.0,0.391144,36.0
